In [1]:
# importing necessary dependencies
import pandas as pd
import datetime as dt
import warnings
import json
import pymongo
warnings.filterwarnings('ignore')

In [2]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'video_game_db' database in Mongo
db = client.video_game_db

# clear all contents
db.destinations.delete_many({})

In [3]:
# Import CSVs

vg_sales = pd.read_csv("data/vgsales.csv")
salesDF = pd.DataFrame(vg_sales)

vg_review = pd.read_csv("data/all_games.csv")
reviewDF = pd.DataFrame(vg_review)

In [4]:
vg_sales.head(5)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [5]:
# renaming name column in vs_review dataset to match salesDF so they can be joined on name column
vg_review.head(5)
vg_review.rename(columns={'name':'Name'}, inplace= True)

In [6]:
# Merge CSVs
DF = pd.merge(salesDF, reviewDF, how= 'inner', on= "Name")
DF.head()

# Drop summary, Platform
DF.drop(columns=['summary', 'Platform'], inplace= True)
DF.head(2)

,Rank,Name,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,platform,release_date,meta_score,user_review
0,1,Wii Sports,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Wii,"November 19, 2006",76,8.1
1,3,Mario Kart Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Wii,"April 27, 2008",82,8.4


In [7]:
# Clean column names
DF.rename(columns={'platform':'Platform', 
                  'release_date':'Release_Date', 
                  'meta_score': 'Meta_Score', 
                  'user_review': 'User_Review'},
          inplace= True)


# Convert Release_Date to datetime
DF['Release_Date'] = DF['Release_Date'].astype('datetime64')
DF.head(2)

# Drop Year column
DF.drop(columns=['Year'], inplace= True)
DF.head(2)


,Rank,Name,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Platform,Release_Date,Meta_Score,User_Review
0,1,Wii Sports,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Wii,2006-11-19,76,8.1
1,3,Mario Kart Wii,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Wii,2008-04-27,82,8.4


In [8]:
# Check for NaN values
# DF.isnull().values.any()
# DF.info()


# Drop Nan values
vg_DF = DF.dropna()
vg_DF.isnull().sum()

Rank            0
Name            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
Platform        0
Release_Date    0
Meta_Score      0
User_Review     0
dtype: int64

In [9]:
# Count duplicates if any
vg_DF.duplicated(keep=False).sum()



# Drop duplicates if any
# none


# Check column types for appropriate dtype
# vg_DF.info()

# Convert User_Review to float
# Drop rows in User_review that has 'tbd'
vg_DF.query('User_Review != "tbd"', inplace= True)



# Convert User_Review to float
# vg_DF['User_Review'].astype('float64')

vg_DF['User_Review'] = vg_DF['User_Review'].astype(float)

In [10]:
# Split release_date column into 3 seperate columns (day, month, year)
vg_DF['Release_Year'] = vg_DF['Release_Date'].dt.year
vg_DF['Release_Month'] = vg_DF['Release_Date'].dt.month
vg_DF['Release_Day'] = vg_DF['Release_Date'].dt.day


vg_DF.head(2)

,Rank,Name,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Platform,Release_Date,Meta_Score,User_Review,Release_Year,Release_Month,Release_Day
0,1,Wii Sports,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Wii,2006-11-19,76,8.1,2006,11,19
1,3,Mario Kart Wii,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Wii,2008-04-27,82,8.4,2008,4,27


In [11]:
# Reorder DF columns as desired


video_gameData = vg_DF[['Rank',
                      'Name', 
                      'Genre', 
                      'Publisher', 
                      'Platform', 
                      'Meta_Score', 
                      'User_Review', 
                      'Release_Date', 
                      'Release_Day', 
                      'Release_Month', 
                      'Release_Year', 
                      'Global_Sales', 
                      'NA_Sales', 
                      'EU_Sales', 
                      'JP_Sales', 
                      'Other_Sales']]
video_gameData

,Rank,Name,Genre,Publisher,Platform,Meta_Score,User_Review,Release_Date,Release_Day,Release_Month,Release_Year,Global_Sales,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Wii Sports,Sports,Nintendo,Wii,76,8.1,2006-11-19,19,11,2006,82.74,41.49,29.02,3.77,8.46
1,3,Mario Kart Wii,Racing,Nintendo,Wii,82,8.4,2008-04-27,27,4,2008,35.82,15.85,12.88,3.79,3.31
2,4,Wii Sports Resort,Sports,Nintendo,Wii,80,8.2,2009-07-26,26,7,2009,33.00,15.75,11.01,3.28,2.96
3,6,Tetris,Puzzle,Nintendo,PSP,85,7.7,2009-10-01,1,10,2009,30.26,23.20,2.26,4.22,0.58
4,6,Tetris,Puzzle,Nintendo,PlayStation 3,80,7.3,2011-01-04,4,1,2011,30.26,23.20,2.26,4.22,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18600,16568,Teslagrad,Platform,Rain Games,Wii U,79,8.1,2014-09-11,11,9,2014,0.01,0.00,0.01,0.00,0.00
18601,16568,Teslagrad,Platform,Rain Games,PlayStation 4,78,6.7,2015-04-14,14,4,2015,0.01,0.00,0.01,0.00,0.00
18602,16568,Teslagrad,Platform,Rain Games,PC,77,7.8,2013-12-13,13,12,2013,0.01,0.00,0.01,0.00,0.00
18603,16582,PGA European Tour,Sports,Infogrames,Nintendo 64,58,8.0,2000-06-15,15,6,2000,0.01,0.01,0.00,0.00,0.00


In [12]:
# convert the final data frame to a dictionary (JSON format) and insert all records into MongoDB
db.collection.insert_many(video_gameData.to_dict('records'))